<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection/blob/main/backbone_fine_tuning_MobileNetv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Testing_set.csv'
test_data = pd.read_csv(test_csv_path)

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

# Load the MobileNet V3 Small model from TensorFlow Hub
model = tfhub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
                         trainable=False,
                         arguments=dict(return_endpoints=True))

# Create an input layer with the desired input shape
input = tf.keras.layers.Input((224, 224, 3))

# Pass the input through the model to obtain the outputs
outputs = model(input)

# Print the names and shapes of the intermediate outputs
for k, v in sorted(outputs.items()):
    print(k, v.shape)

MobilenetV3/global_pool (None, 1, 1, 1280)
MobilenetV3/layer_1 (None, 112, 112, 16)
MobilenetV3/layer_10 (None, 14, 14, 80)
MobilenetV3/layer_10/depthwise_output (None, 14, 14, 184)
MobilenetV3/layer_10/expansion_output (None, 14, 14, 184)
MobilenetV3/layer_10/output (None, 14, 14, 80)
MobilenetV3/layer_11 (None, 14, 14, 80)
MobilenetV3/layer_11/depthwise_output (None, 14, 14, 184)
MobilenetV3/layer_11/expansion_output (None, 14, 14, 184)
MobilenetV3/layer_11/output (None, 14, 14, 80)
MobilenetV3/layer_12 (None, 14, 14, 112)
MobilenetV3/layer_12/depthwise_output (None, 14, 14, 480)
MobilenetV3/layer_12/expansion_output (None, 14, 14, 480)
MobilenetV3/layer_12/output (None, 14, 14, 112)
MobilenetV3/layer_13 (None, 14, 14, 112)
MobilenetV3/layer_13/depthwise_output (None, 14, 14, 672)
MobilenetV3/layer_13/expansion_output (None, 14, 14, 672)
MobilenetV3/layer_13/output (None, 14, 14, 112)
MobilenetV3/layer_14 (None, 7, 7, 160)
MobilenetV3/layer_14/depthwise_output (None, 7, 7, 672)
Mobil

In [ ]:
from tensorflow.keras import layers

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

# Load the MobileNet V3 Small model from TensorFlow Hub
model = tfhub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5',
                         trainable=True,
                         arguments=dict(return_endpoints=True))

# Create an input layer with the desired input shape
input = tf.keras.layers.Input((224, 224, 3))

# Pass the input through the model to obtain the intermediate outputs
outputs = model(input)

# Get the desired layer by name
layer_name = 'MobilenetV3/layer_10/output'
desired_layer = outputs[layer_name]

# Modify the desired layer to have a size of (None, 15)
modified_layer = tf.keras.layers.Conv2D(filters=15, kernel_size=(1, 1))(desired_layer)

# Create a new model with the modified layer as output
new_model = tf.keras.models.Model(inputs=input, outputs=modified_layer)

# Print the shape of the modified layer
print(new_model.output.shape)



(None, 7, 7, 15)


In [ ]:
import tensorflow as tf

# Create a new model with the modified layer as output
new_model = tf.keras.models.Model(inputs=input, outputs=modified_layer)

# Add a GlobalAveragePooling2D layer
pooled_layer = tf.keras.layers.GlobalAveragePooling2D()(new_model.output)

# Add a Dense layer with 15 units and softmax activation
output_layer = tf.keras.layers.Dense(15, activation='softmax')(pooled_layer)

# Create the final model with the modified output layer
final_model = tf.keras.models.Model(inputs=new_model.input, outputs=output_layer)

# Print the summary of the final model
final_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer_1 (KerasLayer)  {'MobilenetV3/global_poo  1529968   
                             l': (None, 1, 1, 1024),             
                              'MobilenetV3/layer_8':             
                             (None, 14, 14, 48),                 
                              'MobilenetV3/layer_4/ou            
                             tput': (None, 28, 28, 24            
                             ),                                  
                              'MobilenetV3/layer_2':             
                             (None, 56, 56, 16),                 
                              'MobilenetV3/layer_6/de            
                             pthwise_output': (None,       

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

import pandas as pd

# Define the target image size and batch size
target_size = (224, 224)
batch_size = 64

# Load the CSV files into dataframes
train_csv = pd.read_csv('/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv')
test_csv = pd.read_csv('/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Testing_set.csv')

# Create ImageDataGenerator with preprocessing options
datagen = ImageDataGenerator(
    rescale=1.0/255.0  # Normalize pixel values between 0 and 1
)

# Define the number of folds for cross-validation
num_folds = 3

# Perform Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)


In [ ]:
from sklearn.model_selection import train_test_split

# Set the number of folds for cross-validation
k = 5

# Iterate through the folds
for fold_num, (train_index, val_index) in enumerate(skf.split(train_csv, train_csv['label']), start=1):
    print("Results for fold", fold_num)

    if train_index.max() < len(train_csv) and val_index.max() < len(train_csv):
        # Split the data into training and validation sets for this fold
        train_fold = train_csv.iloc[train_index]
        val_fold = train_csv.iloc[val_index]

        # Create train and validation generators for this fold
        train_generator = datagen.flow_from_dataframe(
            dataframe=train_fold,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='training'  # Specify training subset for train_generator
        )

        val_generator = datagen.flow_from_dataframe(
            dataframe=val_fold,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='validation'  # Specify validation subset for val_generator
        )

        final_model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(),
            metrics=['accuracy']
        )

        final_model.fit(
            train_generator,
            epochs=15,
            validation_data=val_generator  # Use the validation data during training
        )

        # Evaluate the model on the test set
        test_generator = datagen.flow_from_dataframe(
            dataframe=test_csv,
            directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/test',
            x_col='filename',  # Column name for the filenames
            y_col='label',  # Column name for the labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'
        )

        final_model.evaluate(test_generator)
    else:
        print("Indices out of bounds")


Results for fold 1
Found 11710 validated image filenames belonging to 15 classes.
Found 0 validated image filenames belonging to 15 classes.
Epoch 1/15
183/183 [==============================] - 826s 4s/step - loss: 2.0732 - accuracy: 0.4076
Epoch 2/15
183/183 [==============================] - 687s 4s/step - loss: 1.4323 - accuracy: 0.5823
Epoch 3/15
183/183 [==============================] - 671s 4s/step - loss: 1.1593 - accuracy: 0.6579
Epoch 4/15
183/183 [==============================] - 684s 4s/step - loss: 0.9351 - accuracy: 0.7335
Epoch 5/15
183/183 [==============================] - 689s 4s/step - loss: 0.7550 - accuracy: 0.7907
Epoch 6/15
183/183 [==============================] - 669s 4s/step - loss: 0.5946 - accuracy: 0.8426
Epoch 7/15
183/183 [==============================] - 644s 4s/step - loss: 0.4759 - accuracy: 0.8792
Epoch 8/15
183/183 [==============================] - 647s 4s/step - loss: 0.3738 - accuracy: 0.9159
Epoch 9/15
183/183 [==============================]

KeyboardInterrupt: ignored